# General strategy:
    - identify the bottleneck - with elimination (easiest first)
    - bottleneck can be i.e. the CPU time, reading data from disk, waiting for data, disk space 
    - hardware is not good enough for application (but we need ot improve the bottleneck like that) -> monitor the usage of resources -> identify the bottleneck that is blocking 

## First step 
- open one of those tools and find the bottleneck:

#### Linux - `top`, `iotop`, `iftop`

#### Mac OS - activity monitor

#### Windows - resource monitor, performance monitor

Making things faster:
    
#### - Hierarchy of "slowness": variable currently used (CPU internal memory - fast) < data related to the running programm, but not the currently executed function (RAM - still fast) < file (disk, much slower) << reading over the network (the worse)
- if process requires reading the data from the network maybe try to store it on disk and then read it from disk

- if repeatedly the process is loading smth from disk, to see if it's possible to put info directly in process memory (cash - stores data in form of data that is faster to acess than its original form)
    

Examples of cashed: 

- web-proxy (stored the images that are often view by users behind the proxy)
- DNS - local cahs for the websited they resolve - no need to query for IP adress every time it is requested 
- OS: tries to keep as much info as possible in RAM so we can acess it quickly; = "cashed in memory" - if data is part of the programm that is currently running it will be in RAM -> limited -> OS removes everything that is cached, but not strictly nessessary -> then the ones that are not currently in use in `swap` - onto the hard drive-> can spend a lot of time to read/write from disk with swapping
   - swapping general idea (even though implementation is quite different form one os to another) - info that is not needed rn is removed from RAM and put onto the disk while the information that is needed now is put inot the RAM

What to do if machiene is slow because of swapping:

- `close the ones apps` not needed
- add `more RAM` to the computer
- one of the running programms has a `memory leak`, which causes it to take all of the avaliable memory; `memory leak` - the memory that is not longer needed is not getting released

Some examples:
- if problem goes away with reboot -> computer keeps some state while runnig -> i.e. storing data in memory without deleting it -> runs out of RAM -> change the code that frees the memory used
- slow when starting up -> i.e. lots of the programms starting up with start -> disable the ones that are not needed
- filed that application have grouwn are too large (not solved by reboot) -> bug in the design -> reduce the size of files involves -> if big file are logs, use `Logrotate`
- use subset of users to see if replicates
- hardware failures , i.e. hard drive -> run checks
- malicious software - i.e. malicious browser extensions | scripts to mine for cryptocurrency

In [6]:
# checking 500 requests - and there are ways more options to pass - tests at one time, timeout, etc
# Time per request:       74.102 [ms] (mean)
! ab -n 500 https://google.com/

This is ApacheBench, Version 2.3 <$Revision: 1826891 $>
Copyright 1996 Adam Twiss, Zeus Technology Ltd, http://www.zeustech.net/
Licensed to The Apache Software Foundation, http://www.apache.org/

Benchmarking google.com (be patient)
Completed 100 requests
Completed 200 requests
Completed 300 requests
Completed 400 requests
Completed 500 requests
Finished 500 requests


Server Software:        gws
Server Hostname:        google.com
Server Port:            443
SSL/TLS Protocol:       TLSv1.2,ECDHE-ECDSA-CHACHA20-POLY1305,256,256
TLS Server Name:        google.com

Document Path:          /
Document Length:        220 bytes

Concurrency Level:      1
Time taken for tests:   37.051 seconds
Complete requests:      500
Failed requests:        0
Non-2xx responses:      500
Total transferred:      354000 bytes
HTML transferred:       110000 bytes
Requests per second:    13.49 [#/sec] (mean)
Time per request:       74.102 [ms] (mean)
Time per request:       74.102 [ms] (mean, across all concur

In [9]:
# top if we have the access to the server
! top

>43  mdworker_sha 0.0  00:00.06 3     1    50+   2872K+ 0B     2636K+ 53745outNetworks: packets: 69881756/64G in, 51968905/28G out.616351.100.198.69643597091405538.91.0325+   59441.641 1431+  22M-     3     2    68--   13M            3.197            525267.92, 9.38, 8.795.6829.255.608929103M unused.[951(64) swapins, 98601139(0) swapou218939856112    59920.0   22M   8.56.08

- change the priorities for the processes 0 (highest) to 19 (lowest)
- `nice` - for changing the priority
- `renice` - for the process that is already running
- `pidof` - recieved process name and returns all the process IDs

#### lower the priority of the running process:
```bash
for pid in $(pidof process)name; do renice 19 $pid; done
```

If it did not work (with ab -n 500), see how the processes got started
If it gives to the process still too much time, we can check further: running those CPU-intensive processes in parallel is overloading -> we could modify, what is triggering them and make them run one after the other instead of all runnig at the same time -> let's check how it has started -> ! ps

In [13]:
%%capture
# get info about the processes: all the running processes on the computer
! ps ax
# or to be able to scroll
! ps ax | less

In using `less`: `\` is the search key: i.e. `\process_name` + enter would output the appearences of the processes

`locate` - to see if we can find the file: i.e.
`! locate static/001.webm`

`grep process_name *` - would output the processes connected ot it

`vim` - another command line editor

edit `daemonize` - part that runs the processes in parallel like if daemons -> only one video conversion at a time -> this would not change those processes that are already running ->

` kill -STOP process_name` - stops the processes but does not kill processes completely -> for each command:

- ```bash
for pid in $(pidof process_name); do while kill -CONT $pid; do sleep 1; done; done
```

       -(as there is no command to wait until the process finishes, use while loop)

### More information:

https://docs.microsoft.com/en-us/sysinternals/downloads/procmon 

http://www.brendangregg.com/linuxperf.html

http://brendangregg.com/usemethod.html

Activity Monitor in Mac: 

https://support.apple.com/guide/activity-monitor/welcome/mac

Performance Monitor on Windows: 

https://www.windowscentral.com/how-use-performance-monitor-windows-10

https://www.digitalcitizen.life/how-use-resource-monitor-windows-7

https://docs.microsoft.com/en-us/sysinternals/downloads/process-explorer

https://en.wikipedia.org/wiki/Cache_(computing)

https://www.reddit.com/r/linux/comments/d7hx2c/why_nice_levels_are_a_placebo_and_have_been_for_a/